In [1]:
from sklearn.datasets import fetch_openml
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [2]:
mnist = fetch_openml('mnist_784')

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
x, y = mnist['data'], mnist['target']

In [4]:
x=x/255

In [5]:
x_train, x_test = x[:60000], x[60000:70000]
y_train, y_test = y[:60000], y[60000:70000]


In [6]:
y_train = y_train.astype(np.int8)
y_test = y_test.astype(np.int8)

In [7]:
x_train=x_train.values.reshape(784,60000)
x_test=x_test.values.reshape(784,10000)
y_train=y_train.values.reshape(1,60000)
y_test=y_test.values.reshape(1,10000)

In [8]:
x_train.shape

(784, 60000)

In [9]:
y_train.shape

(1, 60000)

In [10]:
def initialize_parameters_deep(layer_dims):
   np.random.seed(3)
   parameters = {}
   L = len(layer_dims)
   for l in range(1, L):
          parameters['W' + str(l)] =np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
          parameters['b' + str(l)] =np.zeros((layer_dims[l],1))
   return parameters

In [11]:
def linear_forward(A, W, b):
    Z=np.dot(W,A)+b
    cache=(A,W,b)
    return Z, cache

In [12]:
def sigmoid(z):
   s=1/(1+np.exp(-z))
   cache=(z)
   return s,cache

In [13]:
def relu(z):
  s=np.maximum(0,z)
  cache=(z)
  return s,cache

In [14]:
def linear_activation_forward(A_prev, W, b, activation):
     if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache=sigmoid(Z)
     if activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache=relu(Z)
     cache = (linear_cache, activation_cache)

     return A, cache

In [15]:
def L_model_forward(X, parameters):
   caches = []
   A = X
   L = len(parameters) // 2
   for l in range(1,L):
     A_prev=A
     A, cache= linear_activation_forward(A_prev, parameters["W"+str(l)],parameters["b"+str(l)],"relu")
     caches.append(cache)
   AL, cache= linear_activation_forward(A, parameters["W"+str(L)],parameters["b"+str(L)],"sigmoid")
   AL=AL*10
   for i in AL:
    for j in i:
      if j<1:
        j=0
      elif j<2:
        j=1
      elif j<3:
        j=2
      elif j<4:
        j=3
      elif j<5:
        j=4
      elif j<6:
        j=5
      elif j<7:
        j=6
      elif j<8:
        j=7
      elif j<9:
        j=9
   caches.append(cache)

   return AL,caches

In [64]:
def linear_backward(dZ, cache):
   A_prev, W, b = cache
   m = A_prev.shape[1]
   dW=(1/m)*(np.dot(dZ,A_prev.T))
   db=(1/m)*np.sum(dZ,axis=1,keepdims=True)
   dA_prev=np.dot(W.T,dZ)
   return dA_prev,dW,db

In [49]:
def relu_backward(dA,cache,count):
  Z=cache
  count=count+1
  print(count)
  print("shape of Z",Z.shape)
  print("shape of dA",dA.shape)
  x,y=Z.shape
  dZ=np.zeros((x,y))
  for i in range(x):
    for j in range(y):
      if Z[i,j]>0:
        dZ[i,j]=dA[i,j]
  return dZ

In [59]:
def sigmoid_backward(dA,cache,count):
  Z=cache
  A,cache1=sigmoid(Z)
  dZ=A*(1-A)*dA
  return dZ

In [60]:
def linear_activation_backward(dA,Y, cache, activation,count):
  print(dA.shape)
  linear_cache, activation_cache = cache
  if activation == "relu":
   # print(activation_cache)
    dZ=relu_backward(dA,activation_cache,count)
    dA_prev,dW,db=linear_backward(dZ,linear_cache)
  elif activation == "sigmoid":
    dZ=sigmoid_backward(dA,activation_cache,count)
    dA_prev,dW,db=linear_backward(dZ,linear_cache)
    print("sigmoid ",dA_prev.shape)
  return dA_prev, dW, db

In [61]:
def L_model_backward(AL, Y, caches):
  grads = {}
  L = len(caches)
  m = AL.shape[1]
  count=0
  Y = Y.reshape(AL.shape)
  dAL=-(np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
  current_cache = caches[L-1]
  dA_prev_temp, dW_temp, db_temp =linear_activation_backward(dAL,Y,current_cache,"sigmoid",count)
  grads["dA" + str(L-1)] = dA_prev_temp
  grads["dW" + str(L)] = dW_temp
  grads["db" + str(L)] = db_temp
  for l in reversed(range(L-1)):
    current_cache = caches[l]
   # print(current_cache)
    dA_prev_temp, dW_temp, db_temp =linear_activation_backward(grads["dA"+str(l+1)],Y,current_cache,"relu",count)
    grads["dA" + str(l)] = dA_prev_temp
    grads["dW" + str(l+1)] = dW_temp
    grads["db" + str(l+1)] = db_temp
  return grads

In [34]:
def update_parameters(params, grads, learning_rate):
  parameters = params.copy()
  L = len(parameters) // 2
  for l in range(L):
        parameters["W" + str(l+1)] =parameters["W" + str(l+1)]-learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] =parameters["b" + str(l+1)]-learning_rate*grads["db"+str(l+1)]

        # YOUR CODE ENDS HERE
  return parameters

In [35]:
layers_dims = [784, 20, 10, 1]
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000):
  #np.random.seed(1)
  parameters=initialize_parameters_deep(layers_dims)
  for i in range(0, num_iterations):
    AL, caches=L_model_forward(X, parameters)
    grads=L_model_backward(AL, Y, caches)
    parameters=update_parameters(parameters, grads, learning_rate)
  error=AL-Y
  m=Y.shape[1]
  accuracy=100-(10/m)*np.sum(error)
  return accuracy

In [ ]:
print(L_layer_model(x_train,y_train, layers_dims, 0.0075,3000))

Streaming output truncated to the last 5000 lines.
(1, 60000)
sigmoid  (10, 60000)
(10, 60000)
1
shape of Z (10, 60000)
shape of dA (10, 60000)
(20, 60000)
1
shape of Z (20, 60000)
shape of dA (20, 60000)
(1, 60000)
sigmoid  (10, 60000)
(10, 60000)
1
shape of Z (10, 60000)
shape of dA (10, 60000)
(20, 60000)
1
shape of Z (20, 60000)
shape of dA (20, 60000)
(1, 60000)
sigmoid  (10, 60000)
(10, 60000)
1
shape of Z (10, 60000)
shape of dA (10, 60000)
(20, 60000)
1
shape of Z (20, 60000)
shape of dA (20, 60000)
(1, 60000)
sigmoid  (10, 60000)
(10, 60000)
1
shape of Z (10, 60000)
shape of dA (10, 60000)
(20, 60000)
1
shape of Z (20, 60000)
shape of dA (20, 60000)
(1, 60000)
sigmoid  (10, 60000)
(10, 60000)
1
shape of Z (10, 60000)
shape of dA (10, 60000)
(20, 60000)
1
shape of Z (20, 60000)
shape of dA (20, 60000)
(1, 60000)
sigmoid  (10, 60000)
(10, 60000)
1
shape of Z (10, 60000)
shape of dA (10, 60000)
(20, 60000)
1
shape of Z (20, 60000)
shape of dA (20, 60000)
(1, 60000)
sigmoid  (10, 